In [1]:
import sys

In [2]:
sys.path.append('../')

In [3]:
!gcc --version

gcc (Ubuntu 4.8.5-4ubuntu2) 4.8.5
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
!which nvcc

/usr/local/cuda-7.0/bin/nvcc


In [5]:
import cPickle as pickle
import re
import glob
import os

import time

import theano
import theano.tensor as T
import numpy as np
import pandas as p
import lasagne as nn

from utils import hms, architecture_string, get_img_ids_from_iter

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)
/users/avanti/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
%pylab inline

rcParams['figure.figsize'] = 16, 6
# rcParams['text.color'] = 'red'
# rcParams['xtick.color'] = 'red'
# rcParams['ytick.color'] = 'red'

Populating the interactive namespace from numpy and matplotlib


In [7]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

First we load the dump of the trained network.

In [8]:
dump_path = '../dumps/2015_07_17_123003.pkl'

In [9]:
model_data = pickle.load(open(dump_path, 'r'))

In [10]:
# Let's set the in and output layers to some local vars.
l_out = model_data['l_out']
l_ins = model_data['l_ins']

In [11]:
#create mirror non-dnn model

Some info about the architecture of the model:

In [12]:
model_arch = architecture_string(model_data['l_out'])

num_params = nn.layers.count_params(l_out)
model_arch += "\nNumber of parameters: %d.\n\n" % num_params

/users/avanti/Lasagne/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


Setting up some Theano / Lasagne things to get some predictions.

In [13]:
chunk_size = model_data['chunk_size'] * 2
batch_size = model_data['batch_size']

print "Batch size: %i." % batch_size
print "Chunk size: %i." % chunk_size

Batch size: 64.
Chunk size: 256.


In [14]:
output = nn.layers.get_output(l_out, deterministic=True)
nondeterministic_output = nn.layers.get_output(l_out, deterministic=False)
input_ndims = [len(nn.layers.get_output_shape(l_in))
               for l_in in l_ins]
xs_shared = [nn.utils.shared_empty(dim=ndim)
             for ndim in input_ndims]
idx = T.lscalar('idx')

givens = {}
for l_in, x_shared in zip(l_ins, xs_shared):
    givens[l_in.input_var] = x_shared[idx * batch_size:(idx + 1) * batch_size]

compute_output = theano.function(
    [idx],
    output,
    givens=givens,
    on_unused_input='ignore'
)

compute_nondeterministic_output = theano.function(
    [idx],
    nondeterministic_output,
    givens=givens,
    on_unused_input='ignore'
)

In [15]:
# Do transformations per patient instead?
if 'paired_transfos' in model_data:
    paired_transfos = model_data['paired_transfos']
else:
    paired_transfos = False
    
print paired_transfos

True


We're going to test on some train images, so loading the training set labels.

In [16]:
train_labels = p.read_csv(os.path.join('/srv/scratch/avanti/diabetic_retinopathy/trainLabels.csv'))

In [17]:
image_to_label = dict(zip(train_labels.image, train_labels.level))

In [18]:
new_dataloader_params = model_data['data_loader_params']
train_image_ids = set(new_dataloader_params['images_train_0'])
# Get validation set ids
all_patient_ids = sorted(set(get_img_ids_from_iter(train_labels.image)))
valid_patient_ids = [patient_id for patient_id in all_patient_ids
                     if patient_id not in train_image_ids]

In [19]:
valid_patient_ids_labels = [image_to_label[str(patient_id)+"_"+side]
                            for patient_id in valid_patient_ids
                            for side in ["left","right"]]

In [ ]:
valid_labels_fh = open("valid_labels.txt",'w')
for patient_id in valid_patient_ids:
    for side in ["left", "right"]:
        patient_id_and_side = str(patient_id)+"_"+side
        valid_labels_fh.write(
            patient_id_and_side+"\t"+
            "\t".join(str(image_to_label[patient_id_and_side])+"\n"))
valid_labels_fh.close()
!gzip valid_labels.txt

In [20]:
num_chunks = int(np.ceil((2 * len(valid_patient_ids)) / float(chunk_size)))

In [21]:
# Where all the images are located: 
# it looks for [img_dir]/[patient_id]_[left or right].jpeg
img_dir = '/srv/scratch/avanti/diabetic_retinopathy/unzipped_train_ds2_crop/'

Using the DataLoader to set up the parameters, you could replace it with something much simpler.

In [22]:
from generators import DataLoader

In [23]:
data_loader = DataLoader()
#new_dataloader_params.update({'images_test': patient_ids})
#new_dataloader_params.update({'labels_test': train_labels.level.values})
new_dataloader_params.update({'prefix_train': img_dir})
data_loader.set_params(new_dataloader_params)

In [24]:
def do_pred(test_gen, deterministic):
    outputs = []

    for e, (xs_chunk, chunk_shape, chunk_length) in enumerate(test_gen()):
        num_batches_chunk = int(np.ceil(chunk_length / float(batch_size)))

        print "Chunk %i/%i" % (e + 1, num_chunks)

        print "  load data onto GPU"
        for x_shared, x_chunk in zip(xs_shared, xs_chunk):
            x_shared.set_value(x_chunk)

        print "  compute output in batches"
        outputs_chunk = []
        for b in xrange(num_batches_chunk):
            if (deterministic):
                out = compute_output(b)
            else:
                out = compute_nondeterministic_output(b)
            outputs_chunk.append(out)

        outputs_chunk = np.vstack(outputs_chunk)
        outputs_chunk = outputs_chunk[:chunk_length]

        outputs.append(outputs_chunk)

    return np.vstack(outputs), xs_chunk

In [27]:
data_loader.p_x

512

In [42]:
from generators import patches_gen_fixed_pairs


def create_fixed_gen(data_loader, images, chunk_size,
                     prefix_train, prefix_test,
                     transfo_params=None,
                     paired_transfos=paired_transfos):

    if not transfo_params:
        raise ValueError("Need transfo_params for gen!")

    gen = patches_gen_fixed_pairs(
            images, p_x=data_loader.p_x,
            p_y=data_loader.p_y,
            num_channels=data_loader.num_channels,
            chunk_size=chunk_size,
            prefix_train=prefix_train,
            prefix_test=prefix_test,
            transfo_params=transfo_params,
            paired_transfos=paired_transfos)

    def fixed_gen():
        for chunk_x, chunk_dim, chunk_shape, chunk_length in gen:
            yield [(chunk_x - data_loader.zmuv_mean) /
                   (0.05 + data_loader.zmuv_std),
                   chunk_dim], chunk_shape, chunk_length
    return fixed_gen()

In [ ]:
no_transfo_params = model_data['data_loader_params']['no_transfo_params']

transfo_param_sets = [
    {'flip': False,
     'rotation_before_resize': True,
     'rotation_range': (0,1)},
    {'flip': True,
     'flip_prob': 1.0,
     'rotation_before_resize': True,
     'rotation_range': (0,1)},
    {'flip': False,
     'rotation_before_resize': True,
     'rotation_range': (90,91)},
    {'flip': True,
     'flip_prob': 1.0,
     'rotation_before_resize': True,
     'rotation_range': (90,91)},
    {'flip': False,
     'rotation_before_resize': True,
     'rotation_range': (180,181)},
    {'flip': True,
     'flip_prob': 1.0,
     'rotation_before_resize': True,
     'rotation_range': (180,181)},
    {'flip': False,
     'rotation_before_resize': True,
     'rotation_range': (270,271)},
    {'flip': True,
     'flip_prob': 1.0,
     'rotation_before_resize': True,
     'rotation_range': (270,271)},
]

def print_predictions(output_fh, pred_outputs, patient_ids):
    for i,patient_id in enumerate(patient_ids):
        for side in ['left', 'right']:
            if (side=='left'):
                pred = pred_outputs[i*2]
            else:
                pred = pred_outputs[i*2 + 1]
            output_fh.write(str(patient_id)+"_"+side+"\t"
                            +"\t".join([str(x) for x in pred])+"\n")

for transfo_params in transfo_param_sets:
    transfo_string_summary = (
        "flip-"+str(transfo_params['flip'])
        +"_rotamt-"+str(transfo_params['rotation_range'][0]))
    #transfo_string_summary="notransfo"
    if (os.path.exists(transfo_string_summary)==False):
        os.mkdir(transfo_string_summary)
    full_transfo_params = {}
    full_transfo_params.update(no_transfo_params)
    data_generator = lambda: create_fixed_gen(
                        data_loader=data_loader,
                        images=valid_patient_ids,
                        chunk_size=chunk_size,
                        prefix_train=img_dir,
                        prefix_test=img_dir,
                        transfo_params=full_transfo_params,
                        paired_transfos=paired_transfos)
    print("Doing deterministic predictions")
    sys.stdout.flush()
    pred_outputs, chunk_orig = do_pred(test_gen=data_generator,
                                       deterministic=True)
    output_file = transfo_string_summary+"/deterministic_preds.txt"
    output_fh = open(output_file,'w')
    print_predictions(output_fh=output_fh,
                      pred_outputs=pred_outputs,
                      patient_ids=valid_patient_ids)
    output_fh.close()
    for nondeterministic_run in range(100):
        print("nondet run",nondeterministic_run)
        sys.stdout.flush()
        pred_outputs, chunk_orig = do_pred(test_gen=data_generator,
                                           deterministic=False)
        output_file = (transfo_string_summary+
                       "/nondeterministic_preds_"
                       +str(nondeterministic_run)+".txt")
        output_fh = open(output_file,'w')
        print_predictions(output_fh=output_fh,
                          pred_outputs=pred_outputs,
                          patient_ids=valid_patient_ids)
        output_fh.close()

Doing deterministic predictions


In [ ]:
!ls

Then we can get some predictions.

In [79]:
%%time
outputs_orig, chunk_orig = do_pred(test_gen)

Chunk 1/14
  load data onto GPU
  compute output in batches
Chunk 2/14
  load data onto GPU
  compute output in batches
Chunk 3/14
  load data onto GPU
  compute output in batches
Chunk 4/14
  load data onto GPU
  compute output in batches
Chunk 5/14
  load data onto GPU
  compute output in batches
Chunk 6/14
  load data onto GPU
  compute output in batches
Chunk 7/14
  load data onto GPU
  compute output in batches
Chunk 8/14
  load data onto GPU
  compute output in batches
Chunk 9/14
  load data onto GPU
  compute output in batches
Chunk 10/14
  load data onto GPU
  compute output in batches
Chunk 11/14
  load data onto GPU
  compute output in batches
Chunk 12/14
  load data onto GPU
  compute output in batches
Chunk 13/14
  load data onto GPU
  compute output in batches
Chunk 14/14
  load data onto GPU
  compute output in batches
CPU times: user 3min 38s, sys: 23.9 s, total: 4min 2s
Wall time: 4min 14s


Explore some of the predictions.

In [86]:
from sklearn.metrics import roc_auc_score, average_precision_score

prob_disease = 1-outputs_orig[:,0]
is_diseased = np.array([1 if x > 0 else 0
                        for x in valid_patient_ids_labels])
print(roc_auc_score(y_true=is_diseased,
                    y_score=prob_disease))
print(average_precision_score(y_true=is_diseased,
                              y_score=prob_disease))

0.9113879157475888
0.8729840774273611


In [80]:
from metrics import continuous_kappa

In [82]:
outputs_labels = np.argmax(outputs_orig, axis=1)

kappa_eval = continuous_kappa(
                outputs_labels,
                np.array(valid_patient_ids_labels))

metric, conf_mat, \
    hist_rater_a, hist_rater_b, \
    nom, denom = kappa_eval
    
print 'Kappa %.4f' % metric, '\n'
print conf_mat, '\n'
print nom, '\n'
print nom / nom.sum(), nom.sum()

Kappa 0.8277 

[[2473  151   95    4    3]
 [  42   51   29    0    0]
 [  52   46  293   18    9]
 [   0    0  109   58   17]
 [   2    0   10    6   46]] 

[[ 0.     9.438 23.75   2.25   3.   ]
 [ 2.625  0.     1.812  0.     0.   ]
 [13.     2.875  0.     1.125  2.25 ]
 [ 0.     0.     6.812  0.     1.062]
 [ 2.     0.     2.5    0.375  0.   ]] 

[[0.    0.126 0.317 0.03  0.04 ]
 [0.035 0.    0.024 0.    0.   ]
 [0.174 0.038 0.    0.015 0.03 ]
 [0.    0.    0.091 0.    0.014]
 [0.027 0.    0.033 0.005 0.   ]] 74.875
